## Test cameo cofactor swapping

Read about cameo publication here: 
https://pubs.acs.org/doi/full/10.1021/acssynbio.7b00423

Some cameo code:
https://pythonhosted.org/cameo/apidoc_output/cameo.core.html

Co-factor swap by Mark et al:
https://github.com/27410/group-assignment-2021-Group-69420-lycopene-production-in-S.cerevisiae/blob/main/06_cofactor_swap.ipynb


IMPORTANT: to run this, do it from Freds environment, "thedream". 

OptSwap article about NAD and NADH :D 
https://www.sciencedirect.com/science/article/pii/S1096717614000688


In [22]:
# Load libraries
import numpy as np
from cobra.io import read_sbml_model, write_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra import Reaction, Metabolite
from cobra.core.gene import GPR


#From Marks report
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
#from cobra.io import read_sbml_model
#model = read_sbml_model('models/yeast_lyco.xml')

In [59]:
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary.objective_functions import biomass_product_coupled_yield
from cameo.util import TimeMachine
from cameo.flux_analysis.analysis import flux_variability_analysis as fva

In [23]:
from cameo import load_model

In [73]:
# Loading model
infilename = '../../models/iMM904_progesterone_fixed.xml'
print(f"Loading {infilename}")
model = read_sbml_model(infilename)
#model2 = load_model(infilename)

Loading ../../models/iMM904_progesterone_fixed.xml


In [74]:
# This information is needed to make the model produce progesterone. Otherwise, no progesterone is produced. 

model.add_boundary(model.metabolites.get_by_id('progesterone_c'), type='demand')

Reaction identifier,DM_progesterone_c
Name,progesterone demand
Memory address,0x07f9758b090a0
Stoichiometry,progesterone_c --> progesterone -->
GPR,
Lower bound,0
Upper bound,1000.0


In [75]:
# What is this: optimize the model. How?
Fluxes=model.optimize()
Fluxes

,fluxes,reduced_costs
CITtcp,0.00000,-0.000000e+00
13BGH,0.00000,-4.751566e-02
13BGHe,0.00000,1.040834e-17
13GS,0.32667,-6.938894e-18
16GS,0.00000,-9.503132e-02
...,...,...
R07215,0.00000,-6.612596e-02
R04804,0.00000,-1.332418e+00
R_methylpentanal_t,0.00000,0.000000e+00
SK_methylpentanal_e,0.00000,0.000000e+00


In [88]:
#Choose the reaction that produced progesterone as the objective.
model.objective = model.reactions.R02216

In [98]:
#Calculate the product yield 

py = product_yield(model.reactions.R02216, model.reactions.ECYP11A1)
optswap = CofactorSwapOptimization(model=model, objective_function=py)

In [78]:
optswap

In [99]:
solution = optswap.run()

Starting optimization at Thu, 17 Nov 2022 18:06:19


HBox()

Finished after 00:00:25


In [100]:
solution

,index,targets,fitness


In [ ]:
?optswap

# As suggested in [1]_, flux through a given reaction can sometimes be optimized by swapping complementary
# co-factor. This class implements a search for reactions when swapped improve the given objective. Briefly,
# the approach is to

# - find reactions that have all the targeted co-factor pairs e.g. (nad_c -> nadp_c, nadh_c -> nadph_c)

# - add reactions that have the co-factors swapped and then by a search algorithm switching one off in favor of the
#   other

In [35]:
py = product_yield(model.reactions.R02216, model.reactions.ECYP11A1)
py

In [109]:
swap_optimization = CofactorSwapOptimization(model=model, cofactor_id_swaps=(["nad_c","nadh_c"],["fad_c","fadh2_m"]),objective_function=py)
# VIRKER!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

#swap_optimization = CofactorSwapOptimization(model=model,objective_function=py)
swap_optimization

In [110]:
solution=swap_optimization.run(max_size=10)

Starting optimization at Thu, 17 Nov 2022 18:12:53


/home/trinebera/miniconda3/envs/thedream/lib/python3.9/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:353: RuntimeWarning: invalid value encountered in double_scalars
  return round(product_flux / substrate_flux, config.ndecimals)
/home/trinebera/miniconda3/envs/thedream/lib/python3.9/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:353: RuntimeWarning: invalid value encountered in double_scalars
  return round(product_flux / substrate_flux, config.ndecimals)


HBox()

Finished after 00:00:24


/home/trinebera/miniconda3/envs/thedream/lib/python3.9/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:353: RuntimeWarning: invalid value encountered in double_scalars
  return round(product_flux / substrate_flux, config.ndecimals)
/home/trinebera/miniconda3/envs/thedream/lib/python3.9/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:353: RuntimeWarning: invalid value encountered in double_scalars
  return round(product_flux / substrate_flux, config.ndecimals)
/home/trinebera/miniconda3/envs/thedream/lib/python3.9/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:353: RuntimeWarning: invalid value encountered in double_scalars
  return round(product_flux / substrate_flux, config.ndecimals)
/home/trinebera/miniconda3/envs/thedream/lib/python3.9/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:353: RuntimeWarning: invalid value encountered in double_scalars
  return

In [111]:
solution

,index,targets,fitness
0,0,"(GLUSx, R02216)",0.999999
1,1,"(ALCD2x_copy2, R02216)",0.999999
2,2,"(ALCD25xi, R02216)",0.999999
3,3,"(GAPD, R02216)",0.999999
4,4,"(R02216, LNS14DMx)",0.999999
5,5,"(ALCD2ir, R02216)",0.999999
6,6,"(LNS14DMx, R02216)",0.999999
7,7,"(R02216, C22STDSx)",0.999999
8,8,"(R02216, G5SD2)",0.999999
9,11,"(ALCD23xi, R02216)",0.999999


In [85]:
reactions=solution.data_frame.targets
reactions

0    (R02216, LNS14DMx)
1      (FAS120, R02216)
2      (AASAD2, R02216)
3     (R02216, BTDD_RR)
4    (R02216, ALCD26xi)
5     (ALCD2ir, R02216)
6               (GAPD,)
Name: targets, dtype: object

In [87]:
model.reactions.LNS14DMx

Reaction identifier,LNS14DMx
Name,Cytochrome P450 lanosterol 14 alpha demethylase NAD
Memory address,0x07f9725cea130
Stoichiometry,2.0 h_c + lanost_c + 3.0 nadh_c + 3.0 o2_c --> 44mctr_c + for_c + 4.0 h2o_c + 3.0 nad_c 2.0 H+ + Lanosterol C30H50O + 3.0 Nicotinamide adenine dinucleotide - reduced + 3.0 O2 O2 --> 4 4 dimethylcholesta 8 14 24 trienol C29H46O + Formate + 4.0 H2O H2O + 3.0 Nicotinamide adenine...
GPR,(YNL111C and YHR007C and YKL150W) or (YIL043C and YNL111C and YHR007C)
Lower bound,0.0
Upper bound,999999.0


In [86]:
for t in reactions:
    print("Reactions:")
    for id in t:
        r=model.reactions.get_by_id(id)
        #print(r.name, id+",","Fitness:", solution.data_frame.fitness[id][10])
        s=""
        for rea in range(len(r.reactants)):
            if rea==len(r.reactants)-1:
                s+=r.reactants[rea].name+" ==> "
            else:
                s+=r.reactants[rea].name+" + "
        for p in range(len(r.products)):
            if p==len(r.products)-1:
                s+=r.products[p].name
            else:
                s+=r.products[p].name+" + "
        print(s)
        print()


Reactions:
pregnenolone + Nicotinamide adenine dinucleotide ==> progesterone + H+ + Nicotinamide adenine dinucleotide - reduced

H+ + Lanosterol C30H50O + O2 O2 + Nicotinamide adenine dinucleotide - reduced ==> 4 4 dimethylcholesta 8 14 24 trienol C29H46O + Formate + H2O H2O + Nicotinamide adenine dinucleotide

Reactions:
Decanoate (n-C10:0) + H+ + Malonyl CoA C24H33N7O19P3S + Nicotinamide adenine dinucleotide - reduced ==> CO2 CO2 + Coenzyme A + Dodecanoate (n-C12:0) + H2O H2O + Nicotinamide adenine dinucleotide

pregnenolone + Nicotinamide adenine dinucleotide ==> progesterone + H+ + Nicotinamide adenine dinucleotide - reduced

Reactions:
L 2 Aminoadipate C6H10NO4 + ATP C10H12N5O13P3 + H+ + Nicotinamide adenine dinucleotide - reduced ==> L 2 Aminoadipate 6 semialdehyde C6H11NO3 + AMP C10H12N5O7P + Diphosphate + Nicotinamide adenine dinucleotide

pregnenolone + Nicotinamide adenine dinucleotide ==> progesterone + H+ + Nicotinamide adenine dinucleotide - reduced

Reactions:
pregnenolon

In [8]:
from cameo import fba
%time fba_result = fba(model2)

CPU times: user 125 ms, sys: 0 ns, total: 125 ms
Wall time: 117 ms


In [9]:
fba_result.data_frame

,flux
CITtcp,0.000000e+00
13BGH,0.000000e+00
13BGHe,0.000000e+00
13GS,3.266700e-01
16GS,0.000000e+00
...,...
PYDXO,0.000000e+00
PYK,1.772199e+01
PYNP2r,4.707470e-19
PYR5CDm,1.947497e-17


In [13]:
fba_result.display_on_map("iJO1366.Central metabolism")

Builder(reaction_data={'13GS': 0.32667, 'ACCOAC': 0.114455, 'AGAT_SC': 0.00711, 'AGPRim': 0.04626, 'AGTi': 0.0…

In [11]:
model2

Name,iMM904
Memory address,0x07f96fc3458e0
Number of metabolites,1226
Number of reactions,1577
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


In [12]:
fba_result.display_on_map('iMM904.Central metabolism')

Exception: Could not find the map iMM904.Central metabolism on the server